# Demo for the 2018 BHI & BSN Data Challenge

In [1]:
# Import libraries
from tableone import TableOne
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import psycopg2
import getpass
%matplotlib inline

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'public, mimiciii_demo'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

Password:········


In [8]:
# Run query and assign the results to a Pandas DataFrame
# Requires the icustay_detail view from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/demographics
# And the OASIS score from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/severityscores
query = \
"""
WITH first_icu AS (
    SELECT i.subject_id, i.hadm_id, i.icustay_id, i.gender, i.admittime admittime_hospital, 
    i.dischtime dischtime_hospital, i.los_hospital, i.age, i.admission_type, 
    i.hospital_expire_flag, i.intime intime_icu, i.outtime outtime_icu, i.los_icu
    FROM icustay_detail i
    WHERE i.hospstay_seq = 1
    AND i.icustay_seq = 1
    AND i.age >= 16
)
SELECT f.*, o.icustay_expire_flag, o.oasis, o.oasis_prob
FROM first_icu f
LEFT JOIN oasis o
ON f.icustay_id = o.icustay_id;
"""

data = pd.read_sql_query(query,con)

# Display the first few rows of the data

In [47]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob', 'admitday_hospital',
       'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object', name='variable')

In [ ]:
data.head()

# Add day of week to DataFrame

In [ ]:
data['admitday_hospital'] = data.admittime_hospital.dt.weekday_name
data['dischday_hospital'] = data.dischtime_hospital.dt.weekday_name
data['inday_icu'] = data.intime_icu.dt.weekday_name
data['outday_icu'] = data.outtime_icu.dt.weekday_name
data.head()

# Table 1 summary statistics

In [50]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob', 'admitday_hospital',
       'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object', name='variable')

In [52]:
columns = ['gender', 'los_hospital', 'age', 'admission_type', 'hospital_expire_flag', 
           'los_icu','icustay_expire_flag', 'oasis', 'oasis_prob']

groupby = 'inday_icu'

t = TableOne(data, columns=columns, groupby=groupby)
t.tableone

inday_icu=Friday inday_icu=Monday  \
variable             level                                         
n                                             9               14   
admission_type       ELECTIVE         2 (22.22)         1 (7.14)   
                     EMERGENCY        7 (77.78)       13 (92.86)   
                     URGENT                                        
age                               68.01 (19.56)    70.53 (11.72)   
gender               F                5 (55.56)        6 (42.86)   
                     M                4 (44.44)        8 (57.14)   
hospital_expire_flag 0                5 (55.56)        9 (64.29)   
                     1                4 (44.44)        5 (35.71)   
icustay_expire_flag  0                6 (66.67)       11 (78.57)   
                     1                3 (33.33)        3 (21.43)   
los_hospital                      12.33 (14.92)    15.07 (32.02)   
los_icu                            6.67 (11.59)      3.07 (4.98)   
oasis                             26.11 (10.67)     31.57 (7.10)   
oasis_prob                          0.10 (0.13)      0.13 (0.13)   

                               inday_icu=Saturday inday_icu=Sunday  \
variable             level                                           
n                                              12                7   
admission_type       ELECTIVE                                        
                     EMERGENCY         11 (91.67)        7 (100.0)   
                     URGENT              1 (8.33)                    
age                                 89.13 (67.55)  165.97 (127.22)   
gender               F                  7 (58.33)        7 (100.0)   
                     M                  5 (41.67)                    
hospital_expire_flag 0                   9 (75.0)        5 (71.43)   
                     1                   3 (25.0)        2 (28.57)   
icustay_expire_flag  0                 10 (83.33)        6 (85.71)   
                     1                  2 (16.67)        1 (14.29)   
los_hospital                          6.08 (2.94)      7.00 (8.14)   
los_icu                               2.50 (1.57)      5.00 (8.83)   
oasis                                34.08 (9.92)     36.14 (1.77)   
oasis_prob                            0.19 (0.17)      0.17 (0.03)   

                               inday_icu=Thursday inday_icu=Tuesday  \
variable             level                                            
n                                              21                18   
admission_type       ELECTIVE            1 (4.76)         3 (16.67)   
                     EMERGENCY         20 (95.24)        15 (83.33)   
                     URGENT                                           
age                                107.35 (82.22)     87.98 (54.74)   
gender               F                  13 (61.9)        12 (66.67)   
                     M                   8 (38.1)         6 (33.33)   
hospital_expire_flag 0                 12 (57.14)        14 (77.78)   
                     1                  9 (42.86)         4 (22.22)   
icustay_expire_flag  0                 14 (66.67)        16 (88.89)   
                     1                  7 (33.33)         2 (11.11)   
los_hospital                          6.71 (5.06)      12.28 (7.67)   
los_icu                               4.10 (4.79)       5.11 (6.77)   
oasis                                37.81 (7.59)      34.22 (6.07)   
oasis_prob                            0.24 (0.17)       0.16 (0.12)   

                               inday_icu=Wednesday isnull  
variable             level                                 
n                                               18         
admission_type       ELECTIVE             1 (5.56)      0  
                     EMERGENCY          16 (88.89)         
                     URGENT               1 (5.56)         
age                                  63.87 (17.61)      0  
gender               F                   5 (27.78)     

In [41]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'icustay_expire_flag', 'oasis', 'oasis_prob', 'admitday_hospital',
       'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object', name='variable')